# MLflow Core Concepts

In this notebook you will build a model to predict the score quality of a wine given some physicochemical measurements. See [Cortez et al., 2009](http://www3.dsi.uminho.pt/pcortez/wine/) for more detail about the dataset. 

The goal of the notebook is to go through all the different stpes of putting a ML model to productions:
* ingest the data
* split the data for training and evaluation and test
* transform the data for the model
* train and evaluate the model
* store the model
* use the model above to predict on some new data (in batch or real-time)

The goal of this notebook is to give some end-to-end flow. We are not trying to go very deep in any steps but show the overall flow. 

Along this notebook you will have some tasks that need to be completed. You will be able to find where they are in the code by searching for `# ToDo#: ...`

In this notebooks you will be asked to:
* ToDo1: add a column to the data frame that indicates if the wine is red or white
* ToDo2: separate the target variable from the features
* ToDo3: fit the preprocessing pipeline on the training data and transform the validation and test data
* ToDo4: log the model and the preprocessing pipeline
* ToDo5: log metrics to mlflow
* ToDo6: log parameters to mlflow
* ToDo7: go to see your model logged on mlflow and register the model in the UI and set the model stage to production
* ToDo8: load the model from mlflow and make a prediction on the test data
* ToDo9: set the model uri to the model you just registered
* ToDo10: [To Go Further] rebuild a model using sklearn pipeline, log it to mlflow and deploy a serving endpoint

If you need help you can browse through the following documentation:
* [MLflow](https://mlflow.org/docs/latest/index.html)
* [scikit-learn](https://scikit-learn.org/stable/)
* [pandas](https://pandas.pydata.org/docs/)

In [ ]:
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import RegressorMixin
from sklearn.base import BaseEstimator


In [ ]:
# Note: please change the directory if you are not using a dev container.
# We want to have the working directory to be the src folder in the mlflow-trainng repo
os.chdir("/workspaces/mlflow-training/src")


# setup mlflow to use the same setting than in the recipe in notebook 02
from steps.utils import setup_mlflow

setup_mlflow(
    experiment_name="wine_score_notebook",
)


## Ingest data

In [ ]:
red_df = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
    sep=";",
)

white_df = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
    sep=";",
)
# ToDo1: add a column to the data frame that indicates if the wine is red or white
...

df = pd.concat([red_df, white_df])
df


## Split data

We want to split the data to have the following proportion:
- 80% training
- 10% evaluation
- 10% test

In [ ]:
# ToDo2: separate the target variable from the features
y = ...
X = ...

X_train, X_test_val, y_train, y_test_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test_val, y_test_val, test_size=0.5, random_state=42
)


## Transform data

Apply a preprocessing step to by removing the mean and scaling to unit variance. 

In [ ]:
preprocessing_pipeline = Pipeline(
    [
        (
            "ct",
            ColumnTransformer(
                [
                    (
                        "minmax",
                        StandardScaler(),
                        X_train.columns,
                    ),
                ]
            ),
        )
    ]
)

# ToDo3: fit the preprocessing pipeline on the training data and transform the validation and test data
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
X_val_processed = preprocessing_pipeline.transform(X_val)
X_test_processed = preprocessing_pipeline.transform(X_test)


## Train model

In [ ]:
model = LinearRegression()


In [ ]:
def log_metrics(
    model: RegressorMixin, X: pd.DataFrame, y: pd.Series, suffix: str = "test"
) -> dict:
    """Log model perfomance on dataset"""
    y_pred = model.predict(X)
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    metrics = {
        f"{suffix}.mean_absolute_error": mae,
        f"{suffix}.mean_squared_error": mse,
        f"{suffix}.r2_score": r2,
    }
    # ToDo5: log metrics to mlflow
    ...
    return metrics


In [ ]:
def log_parameters(
    model: BaseEstimator,
) -> dict:
    """Log parameters of interest of the model"""
    model_params = model.get_params()

    # ToDo6: log parameters to mlflow
    ...
    return model_params


In [ ]:

with mlflow.start_run() as run:
    model.fit(X_train_processed, y_train)
    # ToDo4: log the model and the preprocessing pipeline
    ...

    # ToDo5: log metrics to mlflow (see above)
    ...
    # ToDo6: log parameters to mlflow (see above)
    ...

    # Note: we store the run id to be able to retrieve the run later
    mlflow_run_id = run.info.run_id


In [ ]:
print(
    "Please copy the command below in a new terminal on your IDE and let it run until the end of the notebook \n"
)

print("mlflow server \\")
print("    --backend-store-uri sqlite:///src/metadata/mlflow/mlruns.db \\")
print("    --default-artifact-root ./src/metadata/mlflow/mlartifacts \\")
print("    --host 0.0.0.0 \\")
print("    --port 5000")

# ToDo7: go to see your model logged on mlflow and register the model in the UI and set the model stage to production
# Note: mlflow ui by going to http://localhost:5000/ or http://0.0.0.0:5000/ in your browser


## Predict with trained model

### Predict on batch inference

In [ ]:
# ToDo8: load the model from mlflow and make a prediction on the test data
loaded_model = ...
predictions = ...


### Predict in real time

We can also use the mlflow model to do rediction in real-time. To do so we will need to:
1. run an mlflow server to be able to distribute the model (already done above)
2. create a serving enpoint which will pull the model from mlflow server
3. finally we can query our model in real time using `curl`

In [ ]:
print("Please copy the command below in a new terminal on your IDE \n")

print("MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve \\")
print("      --host=0.0.0.0 \\")
print("      --port=5001 \\")
print("      --env-manager=local \\")
# ToDo9: set the model uri to the model you just registered
print("      --model-uri=...")


In [ ]:
print("You can copy the command below on one of your terminal \n")

request_data = pd.DataFrame(X_test_processed).iloc[0:4].to_json(orient="records")
print(
    """curl http://0.0.0.0:5001/invocations -H 'Content-Type: application/json' -d '{"dataframe_records": """
    + request_data
    + """}'"""
)


Congratulation! You made it! 

If you still have some time you can take a big breach and try to help the people around you. 

Or if you like you can try to improve on what you already did and see what could be added 

## To Go Further

You can try to combine the transformer and the predictor together in the same sklearn pipeline. 

In [ ]:
# ToDo10: [To Go Further] rebuild a model using sklearn pipeline, log it to mlflow and deploy a serving endpoint
setup_mlflow(
    experiment_name="wine_score_pipeline_notebook",
)
...
